In [34]:
import numpy as np
from qiskit.quantum_info import average_gate_fidelity
from qiskit import QuantumCircuit, execute, Aer
import matplotlib.pyplot as plt

# Define the noise levels and frequencies to study
noise_levels = np.linspace(0, 0.1, 11)
frequencies = np.linspace(0, 0.1, 11)

# Define the ideal counts
shots = 1000
ideal_counts = {'0': shots/2, '1': shots/2}

# Define the X gate circuit
x_gate = QuantumCircuit(1)
x_gate.x(0)

# Compute the fidelity for each combination of noise level and frequency
fidelity_matrix = np.zeros((len(noise_levels), len(frequencies)))
for i, noise_level in enumerate(noise_levels):
    for j, frequency in enumerate(frequencies):
        # Define the noisy counts
        noisy_counts = add_noise(result_list, noise_level, frequency)
        # Compute the fidelity
        fidelity = average_gate_fidelity(x_gate, noisy_counts, ideal_counts)
        fidelity_matrix[i,j] = fidelity

# Plot the fidelity matrix as a heatmap
fig, ax = plt.subplots()
im = ax.imshow(fidelity_matrix, cmap='viridis')
ax.set_xticks(np.arange(len(frequencies)))
ax.set_yticks(np.arange(len(noise_levels)))
ax.set_xticklabels(frequencies)
ax.set_yticklabels(noise_levels)
ax.set_xlabel('Frequency of noise')
ax.set_ylabel('Noise level')
ax.set_title('Fidelity of X gate')
fig.colorbar(im)
plt.show()

AttributeError: 'Counts' object has no attribute 'results'

In [33]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import depolarizing_error, amplitude_damping_error
from qiskit.circuit.library import U1Gate

def add_noise(results_list, noise_level, frequency):
    # Create a noise model with depolarizing and amplitude damping errors
    depol_prob = noise_level / 3
    phase_prob = noise_level / 3
    damping_prob = noise_level / 3
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(depolarizing_error(depol_prob,1), "depolarizing")
    noise_model.add_all_qubit_quantum_error(amplitude_damping_error(damping_prob), "amplitude_damping")
    # Add phase shift noise to the Rz gates
    for result in results_list:
        for circuit in result.results:
            qubit_list = circuit.header.qubit_sizes[0]
            for qubit in qubit_list:
                noisy_key = f"q{qubit}"
                noisy_value = circuit.data(0).qubits[qubit].name
                instruction_list = circuit.data(qubit)
                for instruction in instruction_list:
                    if instruction.name == "u1":
                        if instruction.params[0] != 0:
                            rz = U1Gate(instruction.params[0] + frequency).to_instruction()
                            phase = amplitude_damping_error(phase_prob)
                            noisy_key = noisy_key[:1] + str(qubit) + noisy_key[2:]
                            noise_model.add_quantum_error(phase, [qubit], [rz])
    return noise_model


In [5]:
backend = Aer.get_backend('qasm_simulator')
shots = 1000
result_list = []

for i in range(shots):
    qc = x_gate.copy()
    qc.measure_all()
    job = execute(qc, backend=backend, shots=1)
    result = job.result().get_counts()
    result_list.append(result)

In [37]:
from qiskit.providers.fake_provider import ConfigurableFakeBackend,FakeArmonkV2,FakeOpenPulse2Q
from qiskit.pulse import Schedule, Play, ConstantPulse

# Define the frequency of the noise you want to add
freq = 0.1

# Load a fake backend
backend = FakePulse2Q()

# Get the qubit frequencies from the backend
qubit_freqs = backend.defaults().qubit_freq_est

# Create a phase shift gate with the frequency
phase_shift = ConstantPulse(0.0, duration=backend.configuration().dt)
phase_shift.add_frequency_instruction(freq, 1.0)

# Apply the phase shift gate to the qubits you want to inject noise into
qubits = [0, 1]
schedules = []
for qubit in qubits:
    schedule = Schedule()
    schedule += Play(phase_shift, backend.get_qubit_lo_freq(qubit))
    schedules.append(schedule)

schedules[0].draw()

ImportError: cannot import name 'ConstantPulse' from 'qiskit.pulse' (C:\Users\Fanuel\anaconda3\envs\AQI\lib\site-packages\qiskit\pulse\__init__.py)